In [13]:
import tensorflow as tf
import keras
import keras_nlp
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers
import pandas as pd
import os
import json

In [4]:
DATA_FOLDER = "/mnt/e/sign-lang-data-2"
# create a vocabulary with GPT2 tokenizer
train = pd.read_csv(os.path.join(DATA_FOLDER, "train.csv"))
train["phrase"].to_csv("text.csv", index=None)

In [6]:
# Créer le tokenizer BPE
tokenizer = Tokenizer(models.BPE())

# Pré-tokenizer qui sera utilisé pour découper votre texte en sous-parties, si nécessaire
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)

# Décodeur qui permettra dµe reproduire le texte d'origine à partir des tokens
tokenizer.decoder = decoders.ByteLevel()

# Entraîner le tokenizer
trainer = trainers.BpeTrainer(special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])
tokenizer.train(files=["text.csv"], trainer=trainer)  # Remplacez "mon_fichier.txt" par votre fichier texte

# Enregistrer le tokenizer
tokenizer.save("monBytePairTokenizer.json")

In [19]:
with open("monBytePairTokenizer.json", "r") as f:
    js_file = json.load(f)

In [22]:
vocab = js_file["model"]["vocab"]
merge = js_file["model"]["merges"]

In [23]:
tokenizer = keras_nlp.tokenizers.BytePairTokenizer(vocab, merge)

In [24]:
tokenizer.tokenize("The quick brown fox jumped")

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([  -1,  477,  690,  369, 1653, 2002,  145, 3232,  156], dtype=int32)>

In [4]:
# Unbatched input.
tokenizer = keras_nlp.models.GPT2Tokenizer.from_preset("gpt2_base_en")
tokenizer("The quick brown fox jumped.")

# Batched input.
print(tokenizer(["The quick brown fox jumped.", "The fox slept."]))

# Detokenization.
print(tokenizer.detokenize(tokenizer("The quick brown fox jumped.")))


# Custom vocabulary.
vocab = {"<|endoftext|>": 0, "a": 4, "Ġquick": 5, "Ġfox": 6}
merges = ["Ġ q", "u i", "c k", "ui ck", "Ġq uick"]
merges += ["Ġ f", "o x", "Ġf ox"]
tokenizer = keras_nlp.models.GPT2Tokenizer(vocabulary=vocab, merges=merges)
tokenizer("a quick fox.")

<tf.RaggedTensor [[464, 2068, 7586, 21831, 11687, 13], [464, 21831, 21256, 13]]>
tf.Tensor(b'The quick brown fox jumped.', shape=(), dtype=string)


<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 4,  5,  6, -1], dtype=int32)>